In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Welcome kagglers**

## **Problem Statement:**
We have to develope tha Machine Learning model to predicate approperiate the House price based on the given features set. 

## **All the Lifecycles of the Data Science Project:**
* Data Analysis
* Feature Engineering
* Feature Selection
* Model Building
* Hyper Parameter Tunning
* Model Evaluation
* Model Deployment
* Model Monitoring

In this notebook we have to done the Data Analysis, Feature Engineering, Feature Selection, Model Building, Hyper parameter tunning, Model Evaluation

In [ ]:
# Importing the necessary library 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder

# Displaying the all the columns of the dataframe
pd.set_option('display.max_columns',None)

In [ ]:
# Reading the dataset
df_train = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
df_test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

In [ ]:
# print the shape of the dataset rows and columns
print(df_train.shape)

In [ ]:
# Print out the top 5 records
df_train.head(5)

### **1. In data analysis find out the following:**
1. Missing values
2. All the numerical variables
3. Distribution of the numerical variables
4. Categorical variables
5. Cardinality of catergorical variables
6. Outliers
7. Relationship between independent and dependent feature(SalePrice)

#### 1. **Missing Values**

In [ ]:
# Here we will check the percentage of nan values present in each features
# 1-step make list of features which has missing values

feature_with_nan = [feature for feature in df_train.columns if df_train[feature].isnull().sum()>1]
for feature in feature_with_nan:
    print(feature, np.round(df_train[feature].isnull().mean(),4),'% missing values')

Since they are many missing values, we need to find the releationship between missing values and saleprice

In [ ]:
# ploting the relationship between sales price and missing values using the bar plot

for feature in feature_with_nan:
    # deep copy the dataset
    df = df_train.copy()
    
    # Let's make a variable that indicates 1 if the observation was missing values
    # 0 for not missing values
    df[feature] = np.where(df[feature].isnull(),1,0)
    
    #Let's calculate the mean salesprice where the information is missing values
    plt.figure(figsize=(10,5))
    df.groupby(feature)['SalePrice'].mean().plot.bar()
    plt.ylabel('Sales price')
    plt.legend(['1 for missing values'])
    plt.show()

Here with the relatinship between the missing values and dependent variable is clearly visible. So we need the replace nan values with something meaningfull values. Which we will do in the feature Engineering

From the above dataset some of the feature like id is not required

In [ ]:
print('Id of the house {}'.format(len(df_train.Id)))

#### 2. **Numerical Variables**

In [ ]:
# List of numerical variables
numerical_features = [feature for feature in df_train.columns if df[feature].dtypes !='O']

print('Number of numerical variables {}'.format(len(numerical_features)))

#Display the numerical features
df_train[numerical_features].head()

##### **Temporal variables(Eg. Datetime variables)**
From the dataset we have 4 year variable. we have extract information from the datetime variables like no of years or not of days. One example in this specific scenario can be difference in years between the year the house was built and the year the house was sold we will perform this analysis in the feature engineering.

In [ ]:
# List of variables that contain year information 
year_feature =[feature for feature in numerical_features if 'Yr' in feature or 'Year' in feature]
print(year_feature)

In [ ]:
# Let's explore the content of these year variables
for feature in year_feature:
    print(feature, df_train[feature].unique())

#### 2. **Distribution of Numerical Variables**

In [ ]:
# Lets analyze the temporal datetime variable
# We will check wheather there is a relation between year the house is sold

df_train.groupby('YrSold')['SalePrice'].median().plot()
plt.xlabel('Year Sold')
plt.ylabel('Median House Price')
plt.title('House Price VS YearSold')
plt.show()

In [ ]:
# Here we will compare the difference between all years feature with the yearSold

for feature in year_feature:
    if feature != 'YrSold':
        df = df_train.copy()
        
        # we will capture the difference between year variable and year of house
        df[feature]=df['YrSold'] - df[feature]
        
        plt.scatter(df[feature],df['SalePrice'])
        plt.xlabel(feature)
        plt.ylabel('SalePrice')
        plt.show()

In [ ]:
# Numerical variables are usally of 2'type
# continous variables and discrete variables

discrete_feature = [feature for feature in numerical_features if len(df[feature].unique()) <25 and feature not in year_feature+['Id']]
print('Discrete vaiables count: {}'.format(len(discrete_feature)))
discrete_feature

In [ ]:
df_train[discrete_feature].head()

In [ ]:
# Lets find the relationship between the and sales price
for feature in discrete_feature:
    df = df_train.copy()
    
    df.groupby(feature)['SalePrice'].median().plot.bar()
    plt.xlabel(feature)
    plt.ylabel('SalePrice')
    plt.title(feature)
    plt.show()

#### **Continous Variable**

In [ ]:
continous_feature = [feature for feature in numerical_features if feature not in discrete_feature+year_feature+['Id']]
print('Number of continous feature {}'.format(len(continous_feature)))

In [ ]:
# Lets analyse the continous value by createing histograms
for feature in continous_feature:
    df = df_train.copy()
    
    df[feature].hist(bins=25)
    plt.xlabel(feature)
    plt.ylabel('Count')
    plt.title(feature)
    plt.show()

In [ ]:
# we will be using logrithmic transformation
for feature in continous_feature:
    df = df_train.copy()
    
    if 0 in df[feature].unique():
        pass
    else:
        df[feature]=np.log(df[feature])
        df['SalePrice']= np.log(df['SalePrice'])
        
        plt.scatter(df[feature],df['SalePrice'])
        plt.xlabel(feature)
        plt.ylabel('SalePrice')
        plt.title(feature)
        plt.show()

### **outliers**

In [ ]:
for feature in continous_feature:
    df = df_train.copy()
    
    if 0 in df_train[feature].unique():
        pass
    else:
        df[feature] = np.log(df[feature])
        df.boxplot(column = feature)
        plt.ylabel(feature)
        plt.title(feature)
        plt.show()

#### 4. **Categorical Variables**

In [ ]:
# List  of categorical variables
categorical_features = [feature for feature in df_train.columns if df_train[feature].dtypes =='O']
categorical_features

In [ ]:
df_train[categorical_features].head()

In [ ]:
for feature in categorical_features:
    print('The feature is {} and number of categories are {}'.format(feature, len(df[feature].unique())))

In [ ]:
# Find the relationship between categorical variables and the dependent feature
for feature in categorical_features:
    df = df_train.copy()
    df.groupby(feature)['SalePrice'].median().plot.bar()
    plt.xlabel(feature)
    plt.ylabel('SalesPrice')
    plt.title(feature)
    plt.title(feature)
    plt.show()

In [ ]:
# replace missing value with a new label
feature_nan = [feature for feature in df_train.columns if df_train[feature].isnull().sum()>1 and df_train[feature].dtypes == 'O']

def replace_cat_feature(df,feature_nan):
    data = df.copy()
    
    data[feature_nan]=data[feature_nan].fillna('Missing')
    return data
df_train = replace_cat_feature(df_train, feature_nan)
df_train[feature_nan].isnull().sum()

In [ ]:
df_train.head()

In [ ]:
# Now lets check for numerical variables the contains missing value
numerical_with_nan = [feature for feature in df_train.columns if df_train[feature].isnull().sum()>1 and df_train[feature].dtypes != 'O']

# We will print the numerical nan variables and percentage of missing value
for feature in numerical_with_nan:
    print('{}: {}% missing values'.format(feature, np.around(df_train[feature].isnull().mean(),4)))

In [ ]:
#repacing the numerical missing
# note when lots of outlier present in the dataset to replace with mode and median

for feature in numerical_with_nan:
    # we will replace by using median since there are outlier
    median_value = df_train[feature].median()
    
    # create a new feature to capture nan values
    df_train[feature+'nan']=np.where(df_train[feature].isnull(),1,0)
    df_train[feature].fillna(median_value,inplace=True)
    
df_train[numerical_with_nan].isnull().sum()

In [ ]:
df_train.head()

In [ ]:
# temporal variables (date time variables)
for feature in ['YearBuilt', 'YearRemodAdd', 'GarageYrBlt']:
    df_train[feature]=df_train['YrSold']-df_train[feature]

In [ ]:
df_train.head()

In [ ]:
df_train[['YearBuilt', 'YearRemodAdd', 'GarageYrBlt']]

### **Numerical Variables**
Since the numerical variables are skewed we will perform log normal distribuation


In [ ]:
df_train.head()

In [ ]:
num_features = ['LotFrontage', 'LotArea','1stFlrSF','GrLivArea','SalePrice']

# convert skewed features into log normal
for feature in num_features:
    df_train[feature]=np.log(df[feature])

In [ ]:
# handling the Nan value of LotFrontage features
df_train['LotFrontage'].fillna(df['LotFrontage'].median(),inplace=True)

In [ ]:
df_train.head()

### **Handling Rare Categorical Feature**

We will remove categorical variables that are present less than 1% of the observation

In [ ]:
categorical_features = [feature for feature in df_train.columns if df_train[feature].dtypes =='O']
categorical_features

In [ ]:
for feature in categorical_features:
    temp = df_train.groupby(feature)['SalePrice'].count()/len(df_train)
    temp_df = temp[temp>0.01].index
    df_train[feature]=np.where(df_train[feature].isin(temp_df),df_train[feature],'Rare_var')

In [ ]:
df_train.head()

In [ ]:
for feature in categorical_features:
    labels_ordered = df_train.groupby([feature])['SalePrice'].mean().sort_values().index
    labels_ordered = {k:i for i,k in enumerate(labels_ordered,0)}
    df_train[feature]=df_train[feature].map(labels_ordered)

## **Feature Scalling**

In [ ]:
feature_scale = [feature for feature in df_train.columns if feature not in ['id','SalePrice']]

scaler = MinMaxScaler()
scaler.fit(df_train[feature_scale])

In [ ]:
# transform the train and test and add on the id and salesprice

x_train  = pd.concat([df_train[['Id','SalePrice']].reset_index(drop = True),
                      pd.DataFrame(scaler.transform(df_train[feature_scale]), columns = feature_scale)],axis=1)


In [ ]:
# capture the dependent feature and independent feature
y = x_train[['SalePrice']]
y_train = y.copy()
x_train = x_train.drop(['Id', 'SalePrice'],axis=1)

In [ ]:
## Apply the feature selection
# first, I specify the Lasso Regression model, and I
# select a suitable alpha (equivalent of penalty)
# The bigger the alpha the less features that will be selected

# Then I use the selectFromModel oject from sklearn, which
# will select the features which coefficients are non-zero

feature_sel_model = SelectFromModel(Lasso(alpha=0.005, random_state=0))
feature_sel_model.fit(x_train, y_train)

In [ ]:
# Let's print the number of total and selected features

#this is how we can make a list of the selected features
selected_feature_Lasso = x_train.columns[(feature_sel_model.get_support())]

# Let's print some stats
print('Total features: {}'.format(x_train.shape[1]))
print('Selected features: {}'.format(len(selected_feature_Lasso)))
print('features with coefficients shrank to zero: {}'.format(np.sum(feature_sel_model.estimator_.coef_ == 0)))

In [ ]:
# feature selectin function
def feature_selection(model,x,y,name):
    feature_sel_model = SelectFromModel(model)
    feature_sel_model.fit(x,y)
    
    #this is how we can make a list of the selected features
    selected_feature = x.columns[(feature_sel_model.get_support())]
    
    #Let's print some stats
    print('Name of the model {}'.format(name))
    print('Total feature: {}'.format(x.shape[1]))
    print('Selected feature: {}'.format(len(selected_feature)))
    return selected_feature

In [ ]:
# feature selction for LinearModel
selected_feature_Linear = feature_selection(LinearRegression(),x_train,y_train,'Linear_Regression')

In [ ]:
# feature selection for RandomForestRegression
selected_feature_RandomForest = feature_selection(RandomForestRegressor(),x_train,y_train,'RandomForestRegressor')

In [ ]:
# feature selection for XBGRegressor
selected_feature_XGBRegressor = feature_selection(XGBRegressor(),x_train,y_train,'XGBRegressor')

## **Model Building**

### **1. Linear Regression**

In [ ]:
# capute the selected feature from the training data and split the data into train and test
x_df = x_train.copy()
x_train = x_train[selected_feature_Linear]
x_train,x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2, random_state=0)

In [ ]:
# fit the linear regression model
model_linear = LinearRegression()
model_linear.fit(x_train, y_train)

In [ ]:
# checking the score of linear model
y_pred = model_linear.predict(x_test)
print('Score of the model {}'.format(np.round(model_linear.score(x_test, y_test),2)))

In [ ]:
# root mean squared error of test se
print('Root mean squared of test set: {}'.format(np.sqrt(mean_squared_error(y_test,y_pred))))

### **2. XGBRegressor**

In [ ]:
# capute the selected feature from the training data and split the data into train and test
x = x_df[selected_feature_XGBRegressor]
x_train,x_test,y_train,y_test = train_test_split(x,y, test_size = 0.2, random_state=0)

In [ ]:
# fit the XGBRegressor
xgb = XGBRegressor()
xgb.fit(x_train, y_train)

In [ ]:
# root mean squared error of test se
print('Root mean squared of test set: {}'.format(np.round(np.sqrt(mean_squared_error(y_test,y_pred)),2)))

### **3. Random Forest Regressor**

In [ ]:
# capute the selected feature from the training data and split the data into train and test
x = x_df[selected_feature_RandomForest]
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=0)

In [ ]:
# import and fit the model
random_reg = RandomForestRegressor()
random_reg.fit(x_train, y_train)

In [ ]:
# root mean squared error on the test set
y_pred = random_reg.predict(x_test)
print('root mean squared error: {}'.format(np.round(np.sqrt(mean_squared_error(y_test,y_pred)),2)))

### **4. Lasso**

In [ ]:
# capture the selected features from the training data and split the data into train and test
x = x_df[selected_feature_Lasso]
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=0)

In [ ]:
# import and fit the model
model_lasso = Lasso(alpha=0.005, random_state=0)
model_lasso.fit(x_train,y_train)

In [ ]:
# root mean squared error on the test set
y_pred = model_lasso.predict(x_test)
print('root mean squared error: {}'.format(np.round(np.sqrt(mean_squared_error(y_test,y_pred)),2)))

We compare the rmse values of the four model. We have to select the RandomForestRegressor

## **Hyperparameter Tunning**

In [ ]:
# split the data into test and train
x = x_df[selected_feature_RandomForest]
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=0)

In [ ]:
# set the hyperparameter grid
param_grid = {
    'bootstrap': [True],
    'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
    'max_features': ['auto', 'sqrt'],
    'min_samples_leaf': [1, 2, 4],
    'min_samples_split': [2, 5, 10],
    'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000,2500,3000]
}

In [ ]:
# fit the RandomizedSerchCV
rfr = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator = rfr, param_distributions = param_grid, n_iter = 100, cv=5, verbose=2, random_state=42, n_jobs=-1)
rf_random.fit(x_train, y_train)

In [ ]:
print('The best parameter selection by the randomizedSearchCV: {}'.format(rf_random.best_params_))

## **Building the final Model**

In [ ]:
# Importing the model and set the hyperparameter
model_rf = RandomForestRegressor(n_estimators =1800, min_samples_split=5, min_samples_leaf=1, max_features='sqrt',max_depth=None, bootstrap=True)

In [ ]:
# fiting the model
model_rf.fit(x_train, y_train)

In [ ]:
# predict the result
y_pred = model_rf.predict(x_test)
print('RMSE value: {}'.format(np.round(np.sqrt(mean_squared_error(y_test, y_pred)))))

## **Model Evaluation**

In [ ]:
# 
y_pred = model_rf.predict(x_test)
print('Mean Absolute Error: {}'.format(np.round(mean_absolute_error(y_test, y_pred),3)))
print('Root Mean Squared error: {}'.format(np.round(np.sqrt(mean_squared_error(y_test, y_pred)),3)))
print('R2 Score: {}'.format(np.round(r2_score(y_test, y_pred))))

## **submission**

In [ ]:
x = df_test[selected_feature_RandomForest]

In [ ]:
# handling the nan value
feature_with_nan = [feature for feature in x.columns if x[feature].isnull().sum()>=1]
for feature in feature_with_nan:
    x[feature].fillna(x[feature].median(),inplace=True)

In [ ]:
# encoding the categorical feature
encoder = LabelEncoder()
x['Neighborhood'] = encoder.fit_transform(x['Neighborhood'])

In [ ]:
# normalize the data
scaler = MinMaxScaler()
x = scaler.fit_transform(x)

In [ ]:
y_pred = model_rf.predict(x)

In [ ]:
my_submission = pd.DataFrame({'Id': df_test.Id, 'SalePrice': y_pred})
# you could use any filename. We choose submission here
my_submission.to_csv('submission.csv', index=False)

# **Thankyou**